# Week 3 Assignment

Importing the necessary modules for the exercise.

In [24]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from IPython.core.display import HTML
!pip install geopy
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from geopy.geocoders import Nominatim

## Part 1 

#### Creating a new dataframe with Postal Code, Borough and Neighbourhood information

Make a request using `requests.get()` method to the Wikipedia page. Store the text from the page in a local variable

In [3]:
req = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
page = req.text

Use BeautifulSoup's `html.parser` to parse the page content

In [4]:
soup = BeautifulSoup(page, 'html.parser')

Find all the tables in the page. There might be many tables and we are interested in only one of the tables.

In [5]:
can_table = soup.find_all("table")
print(len(can_table))

3


In [6]:
HTML(str(can_table[0]))

Postal Code,Borough,Neighbourhood
M1A,Not assigned,Not assigned
M2A,Not assigned,Not assigned
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
M8A,Not assigned,Not assigned
M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
M1B,Scarborough,"Malvern, Rouge"


We see that there are three tables in the page and the first of those seems to be the one we want.

Let us look at the table we extracted to see what other HTML elements need to be filtered out.

In [7]:
can_table[0]

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td

We see that the columns names are enclosed within `<th>` and the values of the different rows and columns (the names of the Boroughs, Neighborhoods and the postal codes) are enclosed withint `<td>` tags.

Let us filter these one by one.

In [8]:
columns = [c for c in can_table[0].find_all('th')]
columns

[<th>Postal Code
 </th>,
 <th>Borough
 </th>,
 <th>Neighbourhood
 </th>]

Let us write a lambda function to replace new line characters with just a space

In [9]:
rm_newline = lambda s: s.replace("\n", "")

In [10]:
columns = [rm_newline(c.get_text()) for c in columns]
columns

['Postal Code', 'Borough', 'Neighbourhood']

Let us do a similar thing for filtering out rows (which are in `<td>`)

In [11]:
rows = [r for r in can_table[0].find_all('td')]
rows

[<td>M1A
 </td>,
 <td>Not assigned
 </td>,
 <td>Not assigned
 </td>,
 <td>M2A
 </td>,
 <td>Not assigned
 </td>,
 <td>Not assigned
 </td>,
 <td>M3A
 </td>,
 <td>North York
 </td>,
 <td>Parkwoods
 </td>,
 <td>M4A
 </td>,
 <td>North York
 </td>,
 <td>Victoria Village
 </td>,
 <td>M5A
 </td>,
 <td>Downtown Toronto
 </td>,
 <td>Regent Park, Harbourfront
 </td>,
 <td>M6A
 </td>,
 <td>North York
 </td>,
 <td>Lawrence Manor, Lawrence Heights
 </td>,
 <td>M7A
 </td>,
 <td>Downtown Toronto
 </td>,
 <td>Queen's Park, Ontario Provincial Government
 </td>,
 <td>M8A
 </td>,
 <td>Not assigned
 </td>,
 <td>Not assigned
 </td>,
 <td>M9A
 </td>,
 <td>Etobicoke
 </td>,
 <td>Islington Avenue, Humber Valley Village
 </td>,
 <td>M1B
 </td>,
 <td>Scarborough
 </td>,
 <td>Malvern, Rouge
 </td>,
 <td>M2B
 </td>,
 <td>Not assigned
 </td>,
 <td>Not assigned
 </td>,
 <td>M3B
 </td>,
 <td>North York
 </td>,
 <td>Don Mills
 </td>,
 <td>M4B
 </td>,
 <td>East York
 </td>,
 <td>Parkview Hill, Woodbine Gardens
 </td>,


Let us replace newline characters with spaces

In [12]:
rows = [rm_newline(r.get_text()) for r in rows]
rows

['M1A',
 'Not assigned',
 'Not assigned',
 'M2A',
 'Not assigned',
 'Not assigned',
 'M3A',
 'North York',
 'Parkwoods',
 'M4A',
 'North York',
 'Victoria Village',
 'M5A',
 'Downtown Toronto',
 'Regent Park, Harbourfront',
 'M6A',
 'North York',
 'Lawrence Manor, Lawrence Heights',
 'M7A',
 'Downtown Toronto',
 "Queen's Park, Ontario Provincial Government",
 'M8A',
 'Not assigned',
 'Not assigned',
 'M9A',
 'Etobicoke',
 'Islington Avenue, Humber Valley Village',
 'M1B',
 'Scarborough',
 'Malvern, Rouge',
 'M2B',
 'Not assigned',
 'Not assigned',
 'M3B',
 'North York',
 'Don Mills',
 'M4B',
 'East York',
 'Parkview Hill, Woodbine Gardens',
 'M5B',
 'Downtown Toronto',
 'Garden District, Ryerson',
 'M6B',
 'North York',
 'Glencairn',
 'M7B',
 'Not assigned',
 'Not assigned',
 'M8B',
 'Not assigned',
 'Not assigned',
 'M9B',
 'Etobicoke',
 'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale',
 'M1C',
 'Scarborough',
 'Rouge Hill, Port Union, Highland Creek',
 'M2C',


Now, we have a flattened out list of all the elements (content) in the table and we want to organize them under the columns. The elements (content) in each row is organized within these three columns. So, we need to group them in groups of three (three = number of columns).

In [13]:
rows = list(zip(*[rows[i::3] for i in range(len(columns))]))
rows

[('M1A', 'Not assigned', 'Not assigned'),
 ('M2A', 'Not assigned', 'Not assigned'),
 ('M3A', 'North York', 'Parkwoods'),
 ('M4A', 'North York', 'Victoria Village'),
 ('M5A', 'Downtown Toronto', 'Regent Park, Harbourfront'),
 ('M6A', 'North York', 'Lawrence Manor, Lawrence Heights'),
 ('M7A', 'Downtown Toronto', "Queen's Park, Ontario Provincial Government"),
 ('M8A', 'Not assigned', 'Not assigned'),
 ('M9A', 'Etobicoke', 'Islington Avenue, Humber Valley Village'),
 ('M1B', 'Scarborough', 'Malvern, Rouge'),
 ('M2B', 'Not assigned', 'Not assigned'),
 ('M3B', 'North York', 'Don Mills'),
 ('M4B', 'East York', 'Parkview Hill, Woodbine Gardens'),
 ('M5B', 'Downtown Toronto', 'Garden District, Ryerson'),
 ('M6B', 'North York', 'Glencairn'),
 ('M7B', 'Not assigned', 'Not assigned'),
 ('M8B', 'Not assigned', 'Not assigned'),
 ('M9B',
  'Etobicoke',
  'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale'),
 ('M1C', 'Scarborough', 'Rouge Hill, Port Union, Highland Creek'),
 ('M

We have all the ingredients in the form of python lists. We can use pandas to convert these lists into a dataframe

In [14]:
df = pd.DataFrame(rows, columns=columns)
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Now, we have the dataframe to play with. We can use pandas to filter out further. First, let us drop all the rows in which Borough is "Not assigned"

In [15]:
not_assigned_indexes = df[df['Borough']=='Not assigned'].index
df.drop(not_assigned_indexes, inplace=True)
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Now, let us merge the rows that have the same postal code (rows grouped by postal code) into one row, collecting the neighborhoods belonging to the postal code into a list (achieved by a join using ',')

In [16]:
df = df.groupby(df['Postal Code']).agg({'Borough': 'first', 'Neighbourhood' : ', '.join}).reset_index()

In [17]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


How many rows are there?

In [18]:
df.shape

(103, 3)

So, our dataset has 103 rows organized under 3 columns

## Part 2

#### Gathering geo-spatial data and augmenting our dataframe with this geo-spatial data

Downloaded the geospatial coordinates csv file. Now, reading the file into a pandas dataframe in order to use for the next exercise.

In [19]:
can_geo_data = pd.read_csv('Geospatial_Coordinates.csv')
can_geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Let us merge this dataframe containing geo-spatial data with the earlier data-frame grouping on the common field "Postal Code"

In [21]:
can_geo_df = pd.merge(df, can_geo_data, left_on=['Postal Code'], right_on=['Postal Code'], how='right')
can_geo_df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


Let's save this dataframe as a csv file if needed for further analysis

In [22]:
can_geo_df.to_csv('Canada_GeoSpatial.csv')

## Part 3

#### Cluster the neighbourhoods in Toronto borough and plot it on a map

Let us filter out the rows that have the string "Toronto" in the Borough name.

In [23]:
toronto_df = can_geo_df[can_geo_df['Borough'].str.contains("Toronto")]
toronto_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Let us get the geo-spatial coordinates for Toronto using geopy's Nominatim

In [25]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
toronto_lat = location.latitude
toronto_long = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(toronto_lat, toronto_long))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Let us now use Folium to get a map of Toronto, using the geo-spatial coordinates identified above.

In [28]:
map_toronto = folium.Map(location=[toronto_lat, toronto_long], zoom_start=11)
map_toronto

Now, let us loop through the different geo-spatial coordinates in our data-frame and get the names of the boroughs and neighbourhoods for each of them and add markers for those points on the map

In [40]:
for lat, lng, pincode, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], 
                                                    toronto_df['Postal Code'], toronto_df['Borough'], 
                                                    toronto_df['Neighbourhood']):
    label = '{}({}): [{}]'.format(borough.upper(), pincode, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
map_toronto

The markers are formatted as follows:
* It shows the Borough name in uppercase 
* It shows the Postal Code within brackets (next to the Borough name)
* It shows all the neighbourhoods within square brackets (as a list)